# About

Convert ssd_mobilenet_v2_coco_2018_03_29 to Core ML

Try using TF1 to convert from saved model to Core ML with given input and output names.
Failed with error:
```
NotImplementedError: Conversion for TF op 'Rank' not implemented.
```

# Download model and setup environment

In [1]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -zxvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

--2021-08-12 07:35:57--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.188.48, 2607:f8b0:4004:835::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.188.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M   209MB/s    in 0.9s    

2021-08-12 07:35:58 (209 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_mo

In [2]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 34.9 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=7816336983578d91ae44a37fe390fdab7204dc642b042947c33accb104d38df7
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [3]:
%tensorflow_version 1.x

import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

TensorFlow 1.x selected.
3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
1.15.2


4.1


# Inspect the TF graph

In [4]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    return graph_def


def load_saved_model(path):
  the_graph = tf.Graph()
  with tf.compat.v1.Session(graph=the_graph) as sess:
    tags = [tf.compat.v1.saved_model.tag_constants.SERVING]
    tf.compat.v1.saved_model.loader.load(sess, tags, path)
  return the_graph


def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))

  print('Exported to', filename)


model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
frozen_graph_file = '/content/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb'

graph_def = load_frozen_graph(frozen_graph_file)
with tf.compat.v1.Session(graph=tf.Graph()) as sess:
  tf.import_graph_def(graph_def, name="")
  the_graph = tf.compat.v1.get_default_graph()
  export_ops_name(the_graph, f'{model_name}_ops.txt')


Exported to ssd_mobilenet_v2_coco_2018_03_29_ops.txt


In [6]:
input_node = "image_tensor"
boxes_output_node = "detection_boxes"
scores_output_node = "detection_scores"


image_input = ct.ImageType(name=input_node,
                           shape=(1, 300, 300, 3),
                           bias=[-1,-1,-1], scale=2./255)

ssd_model = ct.convert(
    frozen_graph_file,
    source='tensorflow',
    inputs=[image_input],
    outputs=[boxes_output_node, scores_output_node]
)

print(ssd_model)

Running TensorFlow Graph Passes:  29%|██▊       | 2/7 [00:00<00:00,  8.89 passes/s]WARNING:root:[Constant Propagation] Skip "dead" tensor: Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond/cond/Switch:0
ERROR:root:[False, None]
ERROR:root:[None, True]
ERROR:root:Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond/cond/switch_f:0
ERROR:root:None
ERROR:root:Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond/cond/switch_t:0
ERROR:root:None
Converting Frontend ==> MIL Ops: 100%|██████████| 5/5 [00:00<00:00, 4703.19 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 26/26 [00:00<00:00, 852.50 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 5/5 [00:00<00:00, 4388.27 ops/s]

Converting Frontend ==> MIL Ops:  89%|████████▊ | 785/885 [00:02<00:00, 387.75 ops/s]


NotImplementedError: ignored